**ResNet**-50 for image recognition

---


Build and train the model
Demonstrate callbacks, learning rate scheduling, plotting utilities
Predict test patterns

Dataset: https://www.kaggle.com/c/detect-pneumonia-fall-2020/data

*Importing dataset from google drive*

In [ ]:
from __future__ import print_function
import keras
import cv2
import csv
import tensorflow as tf
import skimage
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation, Concatenate, \
                                    AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tqdm import tqdm
from skimage.transform import resize
import numpy as np
import pandas as pd
import os as os
import time
from numpy.linalg import inv, norm
from csv import reader
np.set_printoptions(suppress=True, precision=3)
from tensorflow.keras.preprocessing import image
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
os.chdir('/content/gdrive/My Drive/Colab Notebooks/')
print(os.getcwd())
save_dir = os.path.join(os.getcwd(), 'saved_models')
from sklearn.model_selection import train_test_split



Reading Images to Ram **size(150,150)**

In [ ]:
trainData="/content/gdrive/My Drive/Xraydataset/train_images"
training_data=[]


i=0
def create_training_data(i):
  for img in os.listdir(trainData):
    print(i) 
       
    with open('/content/gdrive/My Drive/Xraydataset/labels_train.csv', 'r') as read_obj:
    
      csv_reader = reader(read_obj)
   
      for row in csv_reader:
        
        
        if row[0] == img:
          class_num=row[1]
          break       
          
        
         
          
    eikona = image.load_img(os.path.join(trainData,img), target_size=(150,150)) #eikona = image.load_img(os.path.join(trainData,img), target_size=(300, 300)) 
    
    img_array=image.img_to_array(eikona)
    training_data.append([img_array,class_num])
    i+=1
  
create_training_data(i)

In [ ]:
x_train=[]
y_train=[]

for i in range(len(training_data)):
  x_train.append(training_data[i][0])
  y_train.append(training_data[i][1])
  y_train[i]=int(y_train[i])
training_data.clear()

In [ ]:
x_in=Input(shape=[150,150,3])
x_train=np.asarray(x_train)
x_train=x_train/255.0
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_train=np.asarray(x_train)
y_train=np.asarray(y_train)
t_train = keras.utils.to_categorical(y_train)

In [ ]:
testData="/content/gdrive/My Drive/Xraydataset/test_images"
xtest=[] 
for img in os.listdir(testData):
    eikona = image.load_img(os.path.join(testData,img),target_size=(150, 150)) #(300, 300)) 
    img_array=image.img_to_array(eikona)
    xtest.append(img_array)  
xtest=np.asarray(xtest)
xtest=xtest/255.0
xtest -= x_train_mean
t_test = keras.utils.to_categorical(xtest)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train, t_train, test_size=0.33)

In [ ]:
Implementing Resnest layer

In [ ]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [ ]:
def resnet_v1(input_shape, depth, num_classes=3):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (3)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=2, ### originally: 1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            #x=keras.layers.add(Dropout(0.5))
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU

    x = AveragePooling2D(pool_size=8)(x)#8
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',            # sigmoid?
                    kernel_initializer='he_normal')(y)          #GlorotUniform  he_normal

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    print('Model parameters: {:d}'.format(model.count_params()))
    return model

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    return lr

In [ ]:
'''
Epoch monitoring: Print info at every epoch
'''

class MyCallback(keras.callbacks.Callback):
    tstart = None
    def on_train_begin(self, logs={}):
        return
    
    def on_train_end(self, logs={}):
        return
    
    def on_epoch_begin(self, epoch, logs={}):
        self.tstart = time.time()
        print('epoch:{:03d}'.format(epoch+1), end=', ')
        return
    
    def on_epoch_end(self, epoch, logs={}):
        print('loss:{:8.6f}, acc:{:8.6f},  val_loss:{:8.6f}, val_acc:{:8.6f},  val_acc-acc = {:5.2f}%,  lr:{:0.6f}  [{:0.2f} sec]'.format(
                logs.get('loss'), logs.get('acc'),
                logs.get('val_loss'), logs.get('val_acc'),
                100*(logs.get('val_acc')-logs.get('acc')),
                K.eval(self.model.optimizer.lr),
                time.time()-self.tstart))
        return
    
    def on_batch_begin(self, batch, logs={}):
        return
    
    def on_batch_end(self, batch, logs={}):
        return

In [ ]:
depth=50
input_shape = x_train.shape[1:]
model = resnet_v1(input_shape=input_shape, depth=depth)
    
model.compile(loss="categorical_crossentropy",
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['acc'])
#model.summary()


In [ ]:
plot_model(model, show_shapes=True, dpi=48)

In [ ]:
'''
The Model object has the following attributes
['_uses_inputs_arg', 'inputs', 'outputs', 'name', 'trainable',
'_is_compiled', '_expects_training_arg', '_initial_weights', 'supports_masking', 'optimizer',
'_updates', '_losses', '_per_input_losses', '_per_input_updates', '_layers',
'_outbound_nodes', '_inbound_nodes', '_compute_previous_mask', '_built', '_is_graph_network',
'_input_layers', '_output_layers', '_input_coordinates', '_output_coordinates', '_output_mask_cache',
'_output_tensor_cache', '_output_shape_cache', '_network_nodes', '_nodes_by_depth', '_layers_by_depth',
'input_names', 'output_names', '_feed_input_names', '_feed_inputs', '_feed_input_shapes',
'loss', 'metrics', 'loss_weights', 'sample_weight_mode', 'weighted_metrics',
'loss_functions', '_feed_outputs', '_feed_output_names', '_feed_output_shapes', '_feed_loss_fns',
'targets', '_feed_targets', 'sample_weight_modes', '_feed_sample_weight_modes', 'metrics_names',
'metrics_tensors', 'metrics_updates', 'stateful_metric_names', 'stateful_metric_functions', 'total_loss',
'sample_weights', '_feed_sample_weights', '_function_kwargs', 'train_function', 'test_function',
'predict_function', '_collected_trainable_weights', 'history', 'stop_training'])
'''
for layer in range(len(model._layers)):
    layer_obj = model._layers[layer]
    print('{}'.format(layer_obj.name))

In [ ]:
# Training parameters
batch_size =16#32  # orig paper trained all networks with batch_size=128 me 128 crusharei
epochs = 20#0

# Prepare model model saving directory.
model_name = 'resnet50F1-e{epoch:04d}-loss{loss:.3f}-acc{acc:.3f}-valloss{val_loss:.3f}-valacc{val_acc:.3f}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=True,
    # set each sample mean to 0
    samplewise_center=True,
    # divide inputs by std of dataset
    featurewise_std_normalization=True,#true
    # divide each input by its std
    samplewise_std_normalization=True,
    # apply ZCA whitening
    zca_whitening=False, #true???
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,#05?
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0.,
    # randomly shift images horizontally
    width_shift_range=0.,#0.1
    # randomly shift images vertically
    height_shift_range=0.,#0.1
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0., 
    # set range for random channel shifts
    channel_shift_range=0,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=False,#True
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,#none
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [ ]:

#tf.Variable  v = tf.Variable(np.zeros(shape=shape, dtype=dtype), name=name)
# Fit the model on the batches generated by datagen.flow().(datagen.flow(xtrain, t_train, batch_size=batch_size),   (datagen.flow(xtrain, t_train, batch_size=batch_size)              (xtrain, t_train, batch_size=batch_size
history = model.fit(x_train, y_train, batch_size=batch_size,  
                               validation_data=(x_test, y_test),#xtrain, t_train          xtest, t_test
                               epochs=epochs, verbose=0, workers=4,#verbose=0 workers=4
                               steps_per_epoch = int(x_train.shape[0]/batch_size),class_weight={0: .9837, 1: .5394, 2: 1.0},
                               callbacks=[lr_reducer, lr_scheduler, MyCallback(), checkpoint])#lr_scheduler


# Score trained model.
#scores = model.evaluate(xtest, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

epoch:001, WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1849s vs `on_train_batch_end` time: 0.2699s). Check your callbacks.
loss:0.304990, acc:0.936538,  val_loss:2.077518, val_acc:0.346952,  val_acc-acc = -58.96%,  lr:0.001000  [98.42 sec]

Epoch 00001: val_acc did not improve from 0.77302
epoch:002, loss:0.229475, acc:0.977521,  val_loss:2.283374, val_acc:0.448768,  val_acc-acc = -52.88%,  lr:0.001000  [97.98 sec]

Epoch 00002: val_acc did not improve from 0.77302
epoch:003, loss:0.198916, acc:0.990366,  val_loss:2.018332, val_acc:0.488975,  val_acc-acc = -50.14%,  lr:0.001000  [98.03 sec]

Epoch 00003: val_acc did not improve from 0.77302
epoch:004, loss:0.186410, acc:0.996146,  val_loss:1.335735, val_acc:0.707523,  val_acc-acc = -28.86%,  lr:0.001000  [97.95 sec]

Epoch 00004: val_acc did not improve from 0.77302
epoch:005, loss:0.184342, acc:0.991972,  val_loss:1.701018, val_acc:0.712062,  val_acc-acc = -27.99%,  lr:0.00

NameError: ignored

In [ ]:
y1=model.predict(X_test)
y=np.argmax(y1, axis=-1)
y=np.transpose(y)
print("y1:",y1)

y1: [[0.28  0.394 0.326]
 [0.273 0.399 0.328]
 [0.287 0.407 0.306]
 ...
 [0.272 0.414 0.314]
 [0.279 0.406 0.315]
 [0.28  0.406 0.314]]


In [ ]:
print("y:",y)
#print("x:",x)
import pandas as pd
res = pd.DataFrame(y)
#res.index = X_test.index # its important for comparison
res.columns = ["prediction"]
res.to_csv("/content/prediction_results.csv")


y: [1 1 1 ... 1 1 1]


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.xlabel('Epochs')
plt.ylabel('%')
plt.legend(('acc','val-acc'))
plt.grid(b=True)